In [2]:
import pandas as pd
# import data
songs = pd.read_csv('song_details_lyrics_clean.csv')

In [3]:
test_10_rows=songs[:10]
test_10_rows

,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags
0,One Look,Leo,you could be the love of my life\ntell me im n...,One Look,Leo,156000.0,8973.0,88557.0,"['pop', 'english', 'leodasun']"
1,WWE The Game feat Motörhead,WWE Jim Johnston,NaN,WWE: The Game (Triple H) [feat. Motörhead],WWE & Jim Johnston,0.0,2519.0,9004.0,[]
2,Antonin Dvorak Humoresque,David Garrett,NaN,"Antonin Dvorak, Humoresque",David Garrett,0.0,1778.0,7457.0,"['Classical', 'violin', '4 of 10 stars']"
3,Cradle,Mudvayne,breathe breathe breathe breathe\npush push pus...,Cradle,Mudvayne,314000.0,86366.0,368525.0,"['Nu Metal', 'metal', 'math metal', 'alternati..."
4,Reading YouTube Comments,Drug Church,it glows in the center\nbut they havent invent...,Reading YouTube Comments,Drug Church,172000.0,7399.0,38545.0,[]
5,Tissues,YUNGBLUD,i feel left out\nlike a child\nevery time that...,Tissues,YUNGBLUD,0.0,59233.0,481214.0,"['seen live', 'rock']"
6,The Games We Play,Pusha T,drug dealer benzes with gold diggers in em\nin...,The Games We Play,Pusha T,166000.0,270156.0,2858530.0,"['hardcore hip hop', 'rap', 'Energetic', 'Gang..."
7,Buffalo Replaced,Mitski,mosquitoes can enjoy me i cant go inside\nim s...,Buffalo Replaced,Mitski,153000.0,260613.0,1567117.0,"['singer-songwriter', 'folk rock', 'slowcore',..."
8,GODS,League of Legends NewJeans,\ngogogogogogods\ngogogogogogods\n\n \nayy thi...,GODS,"League of Legends, NewJeans",0.0,10217.0,116258.0,[]
9,Starry Eyes,Mötley Crüe,when she laughs\nshes got the power of a child...,Starry Eyes,Mötley Crüe,304000.0,53135.0,255437.0,"['hard rock', 'Glam Metal', 'heavy metal', 'ha..."


## process profanity

In [9]:
# process profanity
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the CSV file
profanity = pd.read_csv('profanity_en.csv')

# Prepare a set of profanity words for faster lookup
profanity_set = set(profanity['text'].str.lower())

# Function to check for profanity and calculate frequency and density
def analyze_lyrics_for_profanity(lyrics):
    # Normalize the lyrics for comparison
    words = lyrics.lower().split()
    total_words = len(words)
    
    # Find profanity words in the lyrics
    profanity_found = [word for word in words if word in profanity_set]
    
    # Calculate frequency of each profanity word
    profanity_freq = Counter(profanity_found)
    
    # Map profanity words to their categories
    profanity_categories = [profanity[profanity['text'].str.lower() == word]['category_1'].iloc[0] for word in profanity_found]
    
    # Calculate frequency of each category
    category_freq = Counter(profanity_categories)
    
    # Calculate density of profanity words in the lyrics
    profanity_density = len(profanity_found) / total_words if total_words else 0
    
    # Return the top 3 categories
    top_3_categories = category_freq.most_common(3)
    
    return {
        'total_words': total_words,
        'profanity_count': len(profanity_found),
        'profanity_density': profanity_density,
        'top_3_categories': top_3_categories
    }




## Polarity and Subjectivity

In [10]:
# Add Polarity
from textblob import TextBlob

def analyze_sentiment(lyrics):
    blob = TextBlob(lyrics)
    return blob.sentiment

# Analyze sentiment of preprocessed lyrics
#sentiment = analyze_sentiment(lyrics)
#print(f"Sentiment polarity: {sentiment.polarity}, Sentiment subjectivity: {sentiment.subjectivity}")

# Polarity is a float within the range [-1.0, 1.0] where 1 means positive statement and -1 means a negative statement.
#Subjectivity quantifies the amount of personal opinion and factual information contained in the text. The higher subjectivity means that the text contains personal opinion rather than factual information.

## NRCLEX Emotions 

In [11]:
import matplotlib.pyplot as plt
from nrclex import NRCLex

# Emotion Analysis using NRCLex
emotion_scores = { 'fear': 0, 'anger': 0, 'anticip': 0, 'trust': 0, 'surprise': 0, 'positive': 0, 'negative': 0, 'sadness': 0, 'disgust': 0, 'joy': 0 }

def get_emotions(lyrics):
    # Splitting the lyrics into lines for emotion analysis
    lines = lyrics.split('\n')
    for line in lines:
        if line.strip():  # Check if the line is not empty
            text_object = NRCLex(line)
            for emotion in emotion_scores.keys():
                emotion_scores[emotion] += text_object.affect_frequencies.get(emotion, 0)

    # Normalizing the scores
    total_lines = len([line for line in lines if line.strip()])
    for emotion in emotion_scores:
        emotion_scores[emotion] /= total_lines

    # Extracting the top emotion
    top_emotions = sorted(emotion_scores.items(), key=lambda item: item[1], reverse=True)[:2]
    top_emotion_1, top_emotion_2 = top_emotions
    return top_emotions


In [19]:
songs[['profanity_density','top_profanity_cat', 'polarity', 'subjectivity', 'emotion1','emotion1_score', 'emotion2','emotion2_score']]=None

In [13]:
test_10_rows

,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags,profanity_density,top_profanity_cat,polarity,subjectivity,emotion1,emotion1_score,emotion2,emotion2_score
0,One Look,Leo,you could be the love of my life\ntell me im n...,One Look,Leo,156000.0,8973.0,88557.0,"['pop', 'english', 'leodasun']",None,None,None,None,None,None,None,None
1,WWE The Game feat Motörhead,WWE Jim Johnston,NaN,WWE: The Game (Triple H) [feat. Motörhead],WWE & Jim Johnston,0.0,2519.0,9004.0,[],None,None,None,None,None,None,None,None
2,Antonin Dvorak Humoresque,David Garrett,NaN,"Antonin Dvorak, Humoresque",David Garrett,0.0,1778.0,7457.0,"['Classical', 'violin', '4 of 10 stars']",None,None,None,None,None,None,None,None
3,Cradle,Mudvayne,breathe breathe breathe breathe\npush push pus...,Cradle,Mudvayne,314000.0,86366.0,368525.0,"['Nu Metal', 'metal', 'math metal', 'alternati...",None,None,None,None,None,None,None,None
4,Reading YouTube Comments,Drug Church,it glows in the center\nbut they havent invent...,Reading YouTube Comments,Drug Church,172000.0,7399.0,38545.0,[],None,None,None,None,None,None,None,None
5,Tissues,YUNGBLUD,i feel left out\nlike a child\nevery time that...,Tissues,YUNGBLUD,0.0,59233.0,481214.0,"['seen live', 'rock']",None,None,None,None,None,None,None,None
6,The Games We Play,Pusha T,drug dealer benzes with gold diggers in em\nin...,The Games We Play,Pusha T,166000.0,270156.0,2858530.0,"['hardcore hip hop', 'rap', 'Energetic', 'Gang...",None,None,None,None,None,None,None,None
7,Buffalo Replaced,Mitski,mosquitoes can enjoy me i cant go inside\nim s...,Buffalo Replaced,Mitski,153000.0,260613.0,1567117.0,"['singer-songwriter', 'folk rock', 'slowcore',...",None,None,None,None,None,None,None,None
8,GODS,League of Legends NewJeans,\ngogogogogogods\ngogogogogogods\n\n \nayy thi...,GODS,"League of Legends, NewJeans",0.0,10217.0,116258.0,[],None,None,None,None,None,None,None,None
9,Starry Eyes,Mötley Crüe,when she laughs\nshes got the power of a child...,Starry Eyes,Mötley Crüe,304000.0,53135.0,255437.0,"['hard rock', 'Glam Metal', 'heavy metal', 'ha...",None,None,None,None,None,None,None,None


## Feature Engineering

In [20]:
# Conduct FE in one code chunk 
df = songs
for index, row in df.iterrows():
    lyrics = row['lyrics']
    if not lyrics or pd.isna(row['lyrics']):
        continue
    # process profanity
    result_dict = analyze_lyrics_for_profanity(row['lyrics'])
    df.at[index, 'profanity_density'] = result_dict['profanity_density']
    if result_dict['profanity_density'] > 0:
        df.at[index, 'top_profanity_cat'] = result_dict['top_3_categories'][0][0]
    result_dict = None

    # process polarity
    if row['polarity'] == None and row['subjectivity'] == None: 
        sentiment = analyze_sentiment(str(lyrics))
        if sentiment:
            if sentiment.polarity:
                df.at[index, 'polarity'] = round(sentiment.polarity, 5)
            if sentiment.subjectivity:
                df.at[index, 'subjectivity'] = round(sentiment.subjectivity, 5)
    sentiment = None

    #process emotions
    if row['emotion1'] == None and row['emotion2'] == None:
        lyrics = row['lyrics']
        top_2_emotions = get_emotions(str(lyrics))
        df.at[index, 'emotion1'] = top_2_emotions[0][0]
        df.at[index, 'emotion1_score'] = top_2_emotions[0][1]
        df.at[index, 'emotion2'] = top_2_emotions[1][0]
        df.at[index, 'emotion2_score'] = top_2_emotions[1][1]
    top_2_emotions = None
df.to_excel('songs_details_lyrics_FE.xlsx')


## Testing

In [16]:
test_10_rows

,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags,profanity_density,top_profanity_cat,polarity,subjectivity,emotion1,emotion1_score,emotion2,emotion2_score
0,One Look,Leo,you could be the love of my life\ntell me im n...,One Look,Leo,156000.0,8973.0,88557.0,"['pop', 'english', 'leodasun']",0.0,None,-0.00143,0.50357,positive,0.126667,negative,0.07
1,WWE The Game feat Motörhead,WWE Jim Johnston,NaN,WWE: The Game (Triple H) [feat. Motörhead],WWE & Jim Johnston,0.0,2519.0,9004.0,[],None,None,None,None,None,None,None,None
2,Antonin Dvorak Humoresque,David Garrett,NaN,"Antonin Dvorak, Humoresque",David Garrett,0.0,1778.0,7457.0,"['Classical', 'violin', '4 of 10 stars']",None,None,None,None,None,None,None,None
3,Cradle,Mudvayne,breathe breathe breathe breathe\npush push pus...,Cradle,Mudvayne,314000.0,86366.0,368525.0,"['Nu Metal', 'metal', 'math metal', 'alternati...",0.040936,sexual anatomy / sexual acts,-0.28963,0.66382,negative,0.111752,disgust,0.054927
4,Reading YouTube Comments,Drug Church,it glows in the center\nbut they havent invent...,Reading YouTube Comments,Drug Church,172000.0,7399.0,38545.0,[],0.0,None,-0.125,0.34375,negative,0.234639,fear,0.17183
5,Tissues,YUNGBLUD,i feel left out\nlike a child\nevery time that...,Tissues,YUNGBLUD,0.0,59233.0,481214.0,"['seen live', 'rock']",0.0,None,0.03286,0.58913,positive,0.178898,joy,0.143101
6,The Games We Play,Pusha T,drug dealer benzes with gold diggers in em\nin...,The Games We Play,Pusha T,166000.0,270156.0,2858530.0,"['hardcore hip hop', 'rap', 'Energetic', 'Gang...",0.022523,racial / ethnic slurs,0.0855,0.46763,positive,0.15624,joy,0.073203
7,Buffalo Replaced,Mitski,mosquitoes can enjoy me i cant go inside\nim s...,Buffalo Replaced,Mitski,153000.0,260613.0,1567117.0,"['singer-songwriter', 'folk rock', 'slowcore',...",0.008264,bodily fluids / excrement,0.09979,0.58416,positive,0.121527,negative,0.050071
8,GODS,League of Legends NewJeans,\ngogogogogogods\ngogogogogogods\n\n \nayy thi...,GODS,"League of Legends, NewJeans",0.0,10217.0,116258.0,[],0.0,None,0.1033,0.43919,positive,0.134588,joy,0.063146
9,Starry Eyes,Mötley Crüe,when she laughs\nshes got the power of a child...,Starry Eyes,Mötley Crüe,304000.0,53135.0,255437.0,"['hard rock', 'Glam Metal', 'heavy metal', 'ha...",0.0,None,0.30893,0.49018,positive,0.130608,joy,0.085964


In [21]:
df

,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags,profanity_density,top_profanity_cat,polarity,subjectivity,emotion1,emotion1_score,emotion2,emotion2_score
0,One Look,Leo,you could be the love of my life\ntell me im n...,One Look,Leo,156000.0,8973.0,88557.0,"['pop', 'english', 'leodasun']",0.0,None,-0.00143,0.50357,positive,0.129932,negative,0.071952
1,WWE The Game feat Motörhead,WWE Jim Johnston,NaN,WWE: The Game (Triple H) [feat. Motörhead],WWE & Jim Johnston,0.0,2519.0,9004.0,[],None,None,None,None,None,None,None,None
2,Antonin Dvorak Humoresque,David Garrett,NaN,"Antonin Dvorak, Humoresque",David Garrett,0.0,1778.0,7457.0,"['Classical', 'violin', '4 of 10 stars']",None,None,None,None,None,None,None,None
3,Cradle,Mudvayne,breathe breathe breathe breathe\npush push pus...,Cradle,Mudvayne,314000.0,86366.0,368525.0,"['Nu Metal', 'metal', 'math metal', 'alternati...",0.040936,sexual anatomy / sexual acts,-0.28963,0.66382,negative,0.111774,disgust,0.054927
4,Reading YouTube Comments,Drug Church,it glows in the center\nbut they havent invent...,Reading YouTube Comments,Drug Church,172000.0,7399.0,38545.0,[],0.0,None,-0.125,0.34375,negative,0.234642,fear,0.171831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294060,Come Out Ye Black Tans,The Wolfe Tones,\ni was born on a dublin street where the loya...,Come Out Ye Black & Tans,The Wolfe Tones,178000.0,36640.0,241115.0,"['irish', 'silver apples moon', 'folk', 'celti...",0.0,None,0.10397,0.47579,trust,0.124464,negative,0.109407
294061,Pink Sky,Be Good,\n go\n\n \nid tell you to stay\ncause ive b...,Pink Sky (Can I Tell You Something?),Be Good,131000.0,76.0,471.0,[],0.002242,sexual anatomy / sexual acts,0.02654,0.53865,negative,0.176048,positive,0.126093
294062,Sink With You,Underoath,this is my emergency\nlet me feel your apathy\...,Sink With You,Underoath,0.0,15350.0,141388.0,"['metalcore', 'hardcore', 'emo', 'screamo', 'c...",0.0,None,0.40741,0.51204,negative,0.089753,sadness,0.062062
294063,Endseekers,Oh Sleeper,\npry your eyes and behold our captain rally r...,Endseekers,"Oh, Sleeper",186000.0,36782.0,235203.0,"['metalcore', 'post-hardcore', 'Christian Meta...",0.0,None,0.01111,0.4412,positive,0.20936,negative,0.144061


In [3]:
import pandas as pd
songs = pd.read_excel('songs_details_lyrics_FE.xlsx')

In [9]:
filter_song_with_lyrics = songs[songs['lyrics'].notna()]
filter_song_with_lyrics_no_sentiment = filter_song_with_lyrics[filter_song_with_lyrics['polarity'].isna() & filter_song_with_lyrics['subjectivity'].isna()]
filter_song_with_lyrics_no_sentiment

,Unnamed: 0,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags,profanity_density,top_profanity_cat,polarity,subjectivity,emotion1,emotion1_score,emotion2,emotion2_score
320,320,That Night,John Maus,why did you sing to me jennifer\nwhy did you s...,That Night,John Maus,73000.0,26441.0,123800.0,"['haunting', 'folk', 'ambient pop', 'favourite...",0.0,NaN,NaN,NaN,sadness,0.145651,negative,0.121999
327,327,빗 속 뉴욕,SHINee,\nyeah the way she walks the way she talks\nth...,빗 속 뉴욕 (Queen of New York),SHINee,199000.0,15456.0,176700.0,"['k-pop', 'pop', 'dance', 'funky', 'Korean']",0.0,NaN,NaN,NaN,positive,0.061579,joy,0.056040
385,385,SHUT MY EYES,jeebanoff,\n \n \n\nevery time i shut my ...,SHUT MY EYES.,jeebanoff,0.0,1071.0,6332.0,[],0.0,NaN,NaN,NaN,positive,0.008343,joy,0.004759
819,819,Hell On Wings,Phenomena,of darkness now\nthe keeper of\neternal flame\...,Hell On Wings,Phenomena,0.0,1801.0,7200.0,['please let there be my kind of music - heavy...,0.0,NaN,NaN,NaN,negative,0.132642,positive,0.104345
967,967,Не звони,Тимати,\n \nthe number you have dialed has been chang...,Не звони,Тимати,0.0,1476.0,6093.0,"['pederastcore', 'post-kircore', 'anal-driven ...",0.0,NaN,NaN,NaN,positive,0.307443,joy,0.301667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293699,293699,Goodbye Milu,Chinese Football,i know youre everywhere everywhere\n\ni\ni kno...,Goodbye Milu,Chinese Football,418000.0,6613.0,25531.0,[],0.0,NaN,NaN,NaN,positive,0.019728,negative,0.012688
293775,293775,MEOW,SUMIN,\n \n \n \n\n \n \n \n\n \nu...,MEOW,SUMIN,0.0,3611.0,21910.0,['bubblegum bass'],0.0,NaN,NaN,NaN,negative,0.015118,anger,0.014070
293797,293797,Lost,Jerrys Kids,correction is unsaid\nlost in a fog\nthis day ...,Lost,Jerry's Kids,190000.0,9371.0,30585.0,"['hardcore punk', 'punk', 'hardcore']",0.0,NaN,NaN,NaN,negative,0.368243,sadness,0.305968
293870,293870,ihavealongdriveaheadofme,surrenderdorothy,i saw the fire inside your eyes and i knew jus...,ihavealongdriveaheadofme,surrenderdorothy,55000.0,7487.0,35747.0,[],0.0,NaN,NaN,NaN,fear,0.260671,negative,0.061749


In [21]:
df = filter_song_with_lyrics_no_sentiment
for index, row in df.iterrows():
    lyrics = row['lyrics']
    # process polarity pd.isna(df.loc[row_index, 'polarity'])
    if pd.isna(row['polarity']) and pd.isna(row['subjectivity']):
        print(index, "started analysis")
        sentiment = analyze_sentiment(str(lyrics))
        if sentiment:
            if sentiment.polarity:
                df.at[index, 'polarity'] = round(sentiment.polarity, 5)
            if sentiment.subjectivity:
                df.at[index, 'subjectivity'] = round(sentiment.subjectivity, 5)
    sentiment = None
df

320 started analysis
327 started analysis
385 started analysis
819 started analysis
967 started analysis
1087 started analysis
1139 started analysis
1275 started analysis
1289 started analysis
1339 started analysis
1393 started analysis
1590 started analysis
1791 started analysis
1805 started analysis
1846 started analysis
2034 started analysis
2144 started analysis
2216 started analysis
2234 started analysis
2556 started analysis
2778 started analysis
2799 started analysis
2809 started analysis
2865 started analysis
3000 started analysis
3127 started analysis
3215 started analysis
3431 started analysis
3581 started analysis
3599 started analysis
3778 started analysis
3854 started analysis
3935 started analysis
4075 started analysis
4158 started analysis
4173 started analysis
4221 started analysis
4340 started analysis
4369 started analysis
4411 started analysis
4567 started analysis
4574 started analysis
4755 started analysis
4760 started analysis
4793 started analysis
4932 started an

,Unnamed: 0,track_name,artist_name,lyrics,track_name_original,artist_name_original,duration,listeners,playcount,toptags,profanity_density,top_profanity_cat,polarity,subjectivity,emotion1,emotion1_score,emotion2,emotion2_score
320,320,That Night,John Maus,why did you sing to me jennifer\nwhy did you s...,That Night,John Maus,73000.0,26441.0,123800.0,"['haunting', 'folk', 'ambient pop', 'favourite...",0.0,NaN,NaN,NaN,sadness,0.145651,negative,0.121999
327,327,빗 속 뉴욕,SHINee,\nyeah the way she walks the way she talks\nth...,빗 속 뉴욕 (Queen of New York),SHINee,199000.0,15456.0,176700.0,"['k-pop', 'pop', 'dance', 'funky', 'Korean']",0.0,NaN,NaN,NaN,positive,0.061579,joy,0.056040
385,385,SHUT MY EYES,jeebanoff,\n \n \n\nevery time i shut my ...,SHUT MY EYES.,jeebanoff,0.0,1071.0,6332.0,[],0.0,NaN,NaN,NaN,positive,0.008343,joy,0.004759
819,819,Hell On Wings,Phenomena,of darkness now\nthe keeper of\neternal flame\...,Hell On Wings,Phenomena,0.0,1801.0,7200.0,['please let there be my kind of music - heavy...,0.0,NaN,NaN,NaN,negative,0.132642,positive,0.104345
967,967,Не звони,Тимати,\n \nthe number you have dialed has been chang...,Не звони,Тимати,0.0,1476.0,6093.0,"['pederastcore', 'post-kircore', 'anal-driven ...",0.0,NaN,NaN,NaN,positive,0.307443,joy,0.301667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293699,293699,Goodbye Milu,Chinese Football,i know youre everywhere everywhere\n\ni\ni kno...,Goodbye Milu,Chinese Football,418000.0,6613.0,25531.0,[],0.0,NaN,NaN,NaN,positive,0.019728,negative,0.012688
293775,293775,MEOW,SUMIN,\n \n \n \n\n \n \n \n\n \nu...,MEOW,SUMIN,0.0,3611.0,21910.0,['bubblegum bass'],0.0,NaN,NaN,NaN,negative,0.015118,anger,0.014070
293797,293797,Lost,Jerrys Kids,correction is unsaid\nlost in a fog\nthis day ...,Lost,Jerry's Kids,190000.0,9371.0,30585.0,"['hardcore punk', 'punk', 'hardcore']",0.0,NaN,NaN,NaN,negative,0.368243,sadness,0.305968
293870,293870,ihavealongdriveaheadofme,surrenderdorothy,i saw the fire inside your eyes and i knew jus...,ihavealongdriveaheadofme,surrenderdorothy,55000.0,7487.0,35747.0,[],0.0,NaN,NaN,NaN,fear,0.260671,negative,0.061749


In [22]:
df.to_excel('Weird_Lyrics_songs.xlsx')